<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/ais/03_storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ukládání dat

Úkládání dat

- Homogenní datové struktury
- Heterogenní datové struktury
- Operace nad datovými strukturami
  - Filter
  - Map
  - Reduce
- SQL (SQLAlchemy Python)
- GraphQL (Graphene Python)



## Homogenní datové struktury

Homogenní datové struktury jsou charakterizované "stejností". Velmi často jsou spojovány s tabulkami v SQL databázích. Tyto tabulky mají definici, které je společná pro všechny záznamy.

V jazyku Python je ekvivalentem seznam (list), jehož položky jsou slovníky (dictionary), případně instance specifické třídy (class). V případě dictionary se předpokládá, že všechny mají stejný seznam klíčů, což je důležité pro zpracování dat pomocí funkcí, které mohou předpokládat přítomnost hodnot.



## Heterogenní datové struktury

## Operace nad datovými strukturami

## SQL / SQLAlchemy

## GraphQL (Graphene)